In [1]:
%cd ..

import importlib
import json
import math
import os
import socket
import sys
import time


internal_path = os.path.abspath(os.path.join('.'))
sys.path.append(internal_path)
sys.path.append(internal_path + '/datasets')
sys.path.append(internal_path + '/utils')

import datetime
import uuid
from argparse import ArgumentParser

import setproctitle
import torch
import numpy as np
import pandas as pd 
import json
import wandb

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import AutoMinorLocator
from matplotlib.collections import LineCollection



/pub/hofmann-scratch/glanzillo/ded


In [2]:
os.environ['WANDB_API_KEY'] = 'c7314b16e17009f66f3043c0b7968e1142054123'
wandb.login()

api = wandb.Api()
runs = api.runs(path='continually/DataEfficientDistillation', filters={'tags': 'useful'})
print("Loaded", len(runs), "runs")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: glanzillo (continually). Use `wandb login --relogin` to force relogin


Loaded 41 runs


In [3]:
dir(runs)

['QUERY',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_include_sweeps',
 '_load_page',
 '_sweeps',
 'client',
 'convert_objects',
 'cursor',
 'entity',
 'filters',
 'index',
 'last_response',
 'length',
 'more',
 'next',
 'objects',
 'order',
 'per_page',
 'project',
 'update_variables',
 'variables']

In [4]:
dir(runs.objects[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attrs',
 '_base_dir',
 '_entity',
 '_exec',
 '_files',
 '_full_history',
 '_include_sweeps',
 '_repr_html_',
 '_sampled_history',
 '_state',
 '_summary',
 'client',
 'create',
 'delete',
 'dir',
 'display',
 'entity',
 'file',
 'files',
 'history',
 'id',
 'json_config',
 'lastHistoryStep',
 'load',
 'log_artifact',
 'logged_artifacts',
 'name',
 'path',
 'project',
 'save',
 'scan_history',
 'snake_to_camel',
 'state',
 'storage_id',
 'summary',
 'sweep',
 'to_html',
 'update',
 'upload_file',
 'url',
 'use_artifact',
 'used_artifacts',
 'user',
 'username',
 'wait_until_finished']

In [34]:
config = json.loads(runs.objects[10].json_config)
config.keys()

dict_keys(['mse', 'sgd', 'seed', 'alpha', 'debug', 'h_dim', 'device', 'exp_id', 'max_lr', 'min_lr', 'compile', 'dataset', 'mlp_dim', 'n_heads', 'seq_len', 'exp_name', 'head_dim', 'log_path', 'n_layers', 'data_root', 'n_workers', 'use_flash', 'eval_every', 'vocab_size', 'decay_steps', 'logger_type', 'temperature', 'wandb_notes', 'project_path', 'grad_clip_norm', 'log_ckpt_every', 'max_eval_steps', 'eval_batch_size', 'log_grads_every', 'max_train_steps', 'test_batch_size', 'train_batch_size', 'log_metrics_every', 'relative_log_path', 'tokens_per_second', 'wandb_entity_name', 'log_terminal_every', 'wandb_project_name', 'log_alignments_every', 'log_activations_every', 'teacher_checkpoint_path', 'gradient_accumulation_steps'])

In [4]:
config['h_dim']['value']

1024

In [18]:
h = runs.objects[0].history()
h.head()

,_step,config,_runtime,_timestamp,parameter count,parameter count without embedding,eval_batches,eval_tokens,eval_bytes
0,0,"{'_type': 'table-file', 'artifact_path': 'wand...",13,1706087768,NaN,NaN,NaN,NaN,NaN
1,1,NaN,14,1706087769,67680256.0,NaN,NaN,NaN,NaN
2,2,NaN,14,1706087769,NaN,34125824.0,NaN,NaN,NaN
3,3,NaN,26,1706087781,NaN,NaN,500.0,NaN,NaN
4,4,NaN,26,1706087781,NaN,NaN,NaN,4096000.0,NaN


In [33]:
h.columns

Index(['_time/forward', '_eval/top10_acc_over_gpuseconds', '_eval/top10_acc',
       '_train/learning_rate_0', '_eval/normalised_ppl_over_tokens',
       '_train/loss', '_time/total_step',
       '_eval/normalised_ppl_over_gpuseconds', '_eval/top1_acc',
       '_eval/normalised_loss', '_eval/ppl', '_time/backward',
       'x_axes/n_tokens', 'x_axes/gpu_seconds', '_time/tokens_per_second',
       'eval_batches', '_eval/total_loss', '_time/load_batch',
       '_eval/top1_acc_over_tokens', '_runtime', '_eval/normalised_ppl',
       'config', '_eval/normalised_loss_over_gpuseconds',
       '_eval/ppl_over_tokens', '_train/tokens_seen',
       '_eval/top1_acc_over_gpuseconds', '_step', 'eval_tokens',
       '_eval/ppl_over_gpuseconds', 'parameter count',
       '_eval/top10_acc_over_tokens', '_eval/loss',
       'parameter count without embedding',
       '_eval/normalised_loss_over_tokens', '_time/iterations_per_second',
       '_time/train_step', '_time/eval', '_timestamp', 'eval_bytes'],

In [25]:
list(h[6:]['_eval/cka'])

[0.40231186151504517,
 0.5032689571380615,
 0.5224665403366089,
 0.536484956741333,
 0.5498769283294678,
 0.5514689087867737,
 0.5602566003799438,
 0.5614172220230103,
 0.5586612820625305,
 0.5795760750770569,
 0.5739375352859497,
 0.5710100531578064,
 0.5867127776145935,
 0.5773130059242249,
 0.5850902199745178,
 0.591157078742981,
 0.592695415019989,
 0.5914744734764099,
 0.6026270389556885,
 0.5916302800178528,
 0.592689573764801,
 0.5996255874633789,
 0.6005229949951172,
 0.6010473966598511,
 0.6057960391044617,
 0.6048132181167603,
 0.6079581379890442,
 0.6118248701095581,
 0.6088635325431824,
 0.6044564843177795,
 0.6107466220855713,
 0.6121087670326233,
 0.623515784740448,
 0.6232402324676514,
 0.6092463135719299,
 0.6210176944732666,
 0.6167471408843994,
 0.6087719798088074,
 0.6146456599235535,
 0.6195009350776672,
 0.6198665499687195,
 0.616502046585083,
 0.6148177981376648,
 0.6190458536148071,
 0.6199896931648254,
 0.6132553815841675,
 0.6244933009147644,
 0.606181204319000

In [44]:
h['_step'].values[-1]

4999

In [47]:

COLUMNS = ['alpha','seed','student_net','teacher_net','step',
           'tokens_seen','cka_val','cka_train','fa_val','fa_train',
           'val_acc_top1', 'val_acc_top10', 'loss_train', 'ppl_val',
           'temperature', 'total_steps']
res_df = pd.DataFrame(columns=COLUMNS)

skipped = 0
for run in runs.objects:
    h = run.history()[6:] # first rows are for validation

    try:
        config = json.loads(run.json_config)
        total = len(h)
        new_runs = '_eval/cka' in h.columns

        #figuring out teacher architecture
        if config['teacher_checkpoint_path']['value'].split("/")[-1] == "gpt-mini":
            teacher = "mini"
        elif config['teacher_checkpoint_path']['value'].split("/")[-1] == "gpt-medium":
            teacher = "medium"
        else: raise Exception("Unknown teacher.")

        #figuring out student architecture
        if config['h_dim']['value']==512:
            student="mini"
        elif config['h_dim']['value']==1024:
            if config['n_layers']['value']==4: student="mini2"
            else: student="medium"

        try: T = config['temperature']['value']
        except KeyError: T=1 
        sub_df = pd.DataFrame({"alpha":[config['alpha']['value']]*total,
                            "seed":[config['seed']['value']]*total,
                            "total_steps":[h['_step'].values[-1]]*total,
                            "temperature":[T]*total,
                            "student_net":[student]*total,
                            "teacher_net":[teacher]*total,
                            "step":list(h['_step']),
                            "tokens_seen":list(h['_train/tokens_seen']),
                            "cka_val": list(h['_eval/cka']) if new_runs else [-1]*total ,
                            "cka_train":list(h['_train/cka']) if new_runs else [-1]*total,
                            "fa_val":list(h['_eval/fa']) if new_runs else [-1]*total,
                            "fa_train":list(h['_train/fa']) if new_runs else [-1]*total,
                            "val_acc_top1":list(h['_eval/top1_acc']),
                            "val_acc_top10":list(h['_eval/top1_acc']),
                            "loss_train":list(h['_train/loss']),
                            "ppl_val":list(h['_eval/ppl'])
                            })
        
    except KeyError: 
        print("Skipping")
        skipped+=1
        continue # skipping the runs without values
    except IndexError: 
        print("Skipping")
        skipped+=1
        continue # skipping the runs without values

    res_df = pd.concat([res_df,sub_df], ignore_index=True)

print(f"Completion. {skipped} runs skipped.")
res_df

Skipping
Skipping
Completion. 2 runs skipped.


,alpha,seed,student_net,teacher_net,step,tokens_seen,cka_val,cka_train,fa_val,fa_train,val_acc_top1,val_acc_top10,loss_train,ppl_val,temperature,total_steps
0,0,13,mini2,medium,300,19726336.0,0.655841,0.555727,0.000927,-0.001080,0.190045,0.190045,1.865341,125.416468,1,600
1,0,13,mini2,medium,600,39387136.0,0.717754,0.603243,-0.000678,0.004146,0.248359,0.248359,1.337096,74.237271,1,600
2,1,13,mini2,medium,300,19726336.0,0.649685,0.555937,0.001222,0.000004,0.180146,0.180146,4.763060,154.423665,1,600
3,1,13,mini2,medium,600,39387136.0,0.694051,0.609089,-0.000238,0.004516,0.229793,0.229793,4.223670,95.231124,1,600
4,1,13,mini,medium,100,6619136.0,0.530750,0.450695,-1.000000,-1.000000,0.145298,0.145298,5.967178,408.339663,1,1400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13934,0,11,mini2,medium,4600,150765568.0,-1.000000,-1.000000,-1.000000,-1.000000,0.334806,0.334806,0.755326,37.387045,1,4999
13935,0,11,mini2,medium,4700,154042368.0,-1.000000,-1.000000,-1.000000,-1.000000,0.335234,0.335234,0.750485,37.281560,1,4999
13936,0,11,mini2,medium,4800,157319168.0,-1.000000,-1.000000,-1.000000,-1.000000,0.335343,0.335343,0.764336,37.225138,1,4999
13937,0,11,mini2,medium,4900,160595968.0,-1.000000,-1.000000,-1.000000,-1.000000,0.335533,0.335533,0.764285,37.197286,1,4999


In [48]:
logs_path = './logs/results/languini_results'
res_df.to_pickle(logs_path)

In [49]:
pd.read_pickle(logs_path)

,alpha,seed,student_net,teacher_net,step,tokens_seen,cka_val,cka_train,fa_val,fa_train,val_acc_top1,val_acc_top10,loss_train,ppl_val,temperature,total_steps
0,0,13,mini2,medium,300,19726336.0,0.655841,0.555727,0.000927,-0.001080,0.190045,0.190045,1.865341,125.416468,1,600
1,0,13,mini2,medium,600,39387136.0,0.717754,0.603243,-0.000678,0.004146,0.248359,0.248359,1.337096,74.237271,1,600
2,1,13,mini2,medium,300,19726336.0,0.649685,0.555937,0.001222,0.000004,0.180146,0.180146,4.763060,154.423665,1,600
3,1,13,mini2,medium,600,39387136.0,0.694051,0.609089,-0.000238,0.004516,0.229793,0.229793,4.223670,95.231124,1,600
4,1,13,mini,medium,100,6619136.0,0.530750,0.450695,-1.000000,-1.000000,0.145298,0.145298,5.967178,408.339663,1,1400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13934,0,11,mini2,medium,4600,150765568.0,-1.000000,-1.000000,-1.000000,-1.000000,0.334806,0.334806,0.755326,37.387045,1,4999
13935,0,11,mini2,medium,4700,154042368.0,-1.000000,-1.000000,-1.000000,-1.000000,0.335234,0.335234,0.750485,37.281560,1,4999
13936,0,11,mini2,medium,4800,157319168.0,-1.000000,-1.000000,-1.000000,-1.000000,0.335343,0.335343,0.764336,37.225138,1,4999
13937,0,11,mini2,medium,4900,160595968.0,-1.000000,-1.000000,-1.000000,-1.000000,0.335533,0.335533,0.764285,37.197286,1,4999
